In [1]:
import numpy as np
from scipy import linalg as LA
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import numpy.matlib

In [3]:
#mock data #imagine rows are syllables and columns are mice
#change values and/or size of array to see how it effects the eigenvectors/eigenvalues 

#in our mock data we have 4 mice that produce 3 syllables
mockdata = np.array([  
        [5.1,5, 13],
        [5,9,28],
        [4,6,2],
        [5,8,16],])
display(mockdata)

mocksum = np.sum(mockdata,axis=1)
mockdata_norm = mockdata/np.transpose(np.matlib.repmat(mocksum,3,1))
#mockdata = mockdata_norm
#display(mockdata)

#centering the data
mockdata -= np.mean(mockdata, axis = 0)  
#calculate covariance matrix
cov = np.cov(mockdata, rowvar = False)
display(cov)
#get eigenvectors and eigenvalues
evals , evecs = LA.eigh(cov)

#formatting and SORT by eigenvalues (aka sort by how much variance each eigen vector explains)
idx = np.argsort(evals)[::-1]
evecs = evecs[:,idx]
evals = evals[idx]

display(evals)
display(evals/sum(evals)*100)

#this is your data represented as PCS 
mockdata_represented_as_pcs = np.dot(mockdata, evecs)
#display(mockdata_represented_as_pcs)

array([[ 5.1,  5. , 13. ],
       [ 5. ,  9. , 28. ],
       [ 4. ,  6. ,  2. ],
       [ 5. ,  8. , 16. ]])

array([[  0.26916667,   0.26666667,   4.19166667],
       [  0.26666667,   3.33333333,  14.66666667],
       [  4.19166667,  14.66666667, 114.25      ]])

array([1.16307993e+02, 1.48242723e+00, 6.20799178e-02])

array([9.86894575e+01, 1.25786660e+00, 5.26759447e-02])

In [30]:
itertools.product([1,2,3],[1,2])

In [ ]:
#running through some examples
n_mice=4
n_syllables=3

#STatiStiCal SIGniFicAncE
#the cutoff should be interpreted as: while all syllables contribute to the distance between groups mathematically,
#only (cutoff-1)% of unintended (randomly walking) syllables should fall above the line.
#this is not the fdr! In all examples, we draw from a random uniform distribution, however, 
#biologial variables are often drawn from normal or exponential distributions
cutoff=.95
compounds=int(np.ceil(np.log2(1/(1-cutoff))))
comp=[]
for i in range(compounds):
    comp.append((1/n_syllables)/(2*(i+1)))
cutoff=(1/n_syllables) + sum(comp)

In [ ]:
def pca_on_np_array(nparray):
    '''rows are members (mice), columns are variables (syllables)'''
    nparray -= np.mean(nparray, axis = 0)  
    #calculate covariance matrix
    cov = np.cov(nparray, rowvar = False)
    #get eigenvectors and eigenvalues
    evals , evecs = LA.eigh(cov)

    #formatting and SORT by eigenvalues (aka sort by how much variance each eigen vector explains)
    idx = np.argsort(evals)[::-1]
    evecs = evecs[:,idx]
    evals = evals[idx]

    #this is your data represented as PCS 
    nparray_represented_as_pcs = np.dot(nparray, evecs)
    
    return nparray_represented_as_pcs,evecs,evals

In [ ]:
def var_explained(evals):
    var_explained=evals/sum(evals)
    return var_explained

In [ ]:
def pc_dist(nparray1,nparray2):
    '''
    calculate the distance between two groups and within each group
    two groups should have the same number of variables columns
    '''
    
    # first do pca on COMBINED data
    a,b,c=pca_on_np_array(np.concatenate((nparray1,nparray2),axis=0))
    #a=data represented as pc scores rather than variable (syllable) values
    #b=eigenvectors
    #c=eigenvalue corresponding to each eigen vector (use this to calculate variance explained by each pc)
    
    #within array 1 dist
    dist1=[]
    for combos in list(itertools.combinations(a[0:len(nparray1)], 2)):
        #calculate difference by finding difference in each pc and doing squareroot of sum of squares
        dist1.append(sum(np.square(np.subtract(combos[0],combos[1])))**.5)
        #print(sum(np.square(np.subtract(combos[0],combos[1])))**.5)
    
   
    #within array 2 dist
    dist2=[]
    for combos in list(itertools.combinations(a[len(nparray1):len(a)], 2)):
        #calculate difference by finding difference in each pc and doing squareroot of sum of squares
        dist2.append(sum(np.square(np.subtract(combos[0],combos[1])))**.5)
        #print(sum(np.square(np.subtract(combos[0],combos[1])))**.5)
        
        
    #between array 1 and 2 dist
    betweendist=[]
    for combos in list(itertools.product(nparray1, nparray2)):
        betweendist.append(sum(np.square(np.subtract(combos[0],combos[1])))**.5)
        
    return betweendist,dist1,dist2

In [ ]:
def contribution_of_each_syllable_to_distance(nparray1,nparray2):
    pcs,eigenvectors,eigenvalues=pca_on_np_array(np.concatenate((nparray1,nparray2),axis=0))

    n_mice1=len(nparray1)
    n_mice2=len(nparray2)
    n_syllables=len(nparray1[0])

    #get the mean pc scores for each group
    group1_mean_pcs=np.mean(pcs[0:(n_mice1-1)],axis=0)
    group2_mean_pcs=np.mean(pcs[n_mice1:((n_mice1+n_mice2)-1)],axis=0)
    #calculate the absolute pairwise difference between pcs and divide by the sum 
    difference=np.abs(np.subtract(group1_mean_pcs,group2_mean_pcs))
    square_difference=np.square(difference)
    sum_of_square_difference=sum(square_difference)
    weighted_difference=square_difference/sum_of_square_difference

    #calculate how much each syllable contributes to each eigenvector and ultimately the distance between groups
    contribution_of_each_syllable=np.zeros((n_syllables,n_syllables),dtype=float)
    for i in range(n_syllables):
        pc_i_contribution_to_all_syllables=weighted_difference[i]*np.square(eigenvectors[:,i])*((eigenvalues[i]**2/sum(eigenvalues**2)))
        contribution_of_each_syllable[i,:]=pc_i_contribution_to_all_syllables
    
    contribution_of_each_syllable=sum(contribution_of_each_syllable)/sum(sum(contribution_of_each_syllable))
    
    return contribution_of_each_syllable

In [ ]:
def contribution_of_each_syllable_to_distance_raw(nparray1,nparray2):
    '''distance can be calculated without doing pca'''
    
    n_mice1=len(nparray1)
    n_mice2=len(nparray2)
    n_syllables=len(nparray1[0])

    #get the mean pc scores for each group
    group1_mean=np.mean(nparray1,axis=0)
    group2_mean=np.mean(nparray2,axis=0)

    #calculate the absolute pairwise difference between pcs and divide by the sum 
    difference=np.abs(np.subtract(group1_mean,group2_mean))
    square_difference=np.square(difference)
    sum_of_difference=sum(square_difference)
    weighted_difference=square_difference/sum_of_difference

    return weighted_difference

In [ ]:
#example1: two groups, each have n_mice mice and produce n_syllables syllables
#in this example group2 (red) uses it's syllables about twice as much as group 1(+.5) (artificially boosted average)
example1_group1=np.random.rand(n_mice,n_syllables)
example1_group2=np.random.rand(n_mice,n_syllables)+.5
example1_between,example1_within1,example1_within2=pc_dist(example1_group1,example1_group2)

In [ ]:
sns.distplot(example1_between, color='purple')
sns.distplot(example1_within1, color='blue')
sns.distplot(example1_within2, color='red')

In [ ]:
#example2: two groups, each have 20 mice and produce 10 syllables
#in this example group2 (red) uses the first syllable ~ three times as much (~+1) as group 1 does
example2_group1=np.random.rand(n_mice,n_syllables)
example2_group2=np.random.rand(n_mice,n_syllables)
example2_group2[:,2]+=1.5
example2_between,example2_within1,example2_within2=pc_dist(example2_group1,example2_group2)

In [ ]:
f, axes = plt.subplots()
sns.distplot(example2_between, color='purple',label='distance between groups')
sns.distplot(example2_within1, color='blue',label='distance within group 1')
sns.distplot(example2_within2, color='red',label='distance within group 2')
plt.legend()

In [ ]:
#example: two groups' variables (syllables) are drawn from same distribution
example3_group1=np.random.rand(n_mice,n_syllables)
example3_group2=np.random.rand(n_mice,n_syllables)

In [ ]:
f, axes = plt.subplots(1,3)
f.set_size_inches(24,4)

example3_between,example3_within1,example3_within2=pc_dist(example3_group1,example3_group2)
sns.distplot(example3_between, color='purple',label=['distance between groups'],ax=axes[0])
sns.distplot(example3_within1, color='blue',label=['distance within group 1'],ax=axes[0])
sns.distplot(example3_within2, color='red',label=['distance within group 2'],ax=axes[0])
axes[0].set_title('pc distance')
axes[0].legend()

example3=contribution_of_each_syllable_to_distance_raw(example3_group1,example3_group2)
sns.barplot(x=np.array(range(n_syllables)),y=example3,ax=axes[1])
axes[1].set_title('contribution of variable to distance')
axes[1].axhline(cutoff, ls='--')

example3=contribution_of_each_syllable_to_distance(example3_group1,example3_group2)
sns.barplot(x=np.array(range(n_syllables)),y=example3,ax=axes[2])
axes[2].set_title('contribution of variable to distance after pca')
axes[2].axhline(cutoff, ls='--')

In [ ]:
#example: variables (syllables) 2 and 5 are artifically boosted in group 1 vs group 2
example3_group1=np.random.rand(n_mice,n_syllables)
example3_group2=np.random.rand(n_mice,n_syllables)
example3_group2[:,2]+=.5
example3_group2[:,5]+=.75

In [ ]:
f, axes = plt.subplots(1,3)
f.set_size_inches(24,4)

example3_between,example3_within1,example3_within2=pc_dist(example3_group1,example3_group2)
sns.distplot(example3_between, color='purple',label=['distance between groups'],ax=axes[0])
sns.distplot(example3_within1, color='blue',label=['distance within group 1'],ax=axes[0])
sns.distplot(example3_within2, color='red',label=['distance within group 2'],ax=axes[0])
axes[0].set_title('pc distance')
axes[0].legend()

example3=contribution_of_each_syllable_to_distance_raw(example3_group1,example3_group2)
sns.barplot(x=np.array(range(n_syllables)),y=example3,ax=axes[1])
axes[1].set_title('contribution of variable to distance')
axes[1].axhline(cutoff, ls='--')

example3=contribution_of_each_syllable_to_distance(example3_group1,example3_group2)
sns.barplot(x=np.array(range(n_syllables)),y=example3,ax=axes[2])
axes[2].set_title('contribution of variable to distance after pca')
axes[2].axhline(cutoff, ls='--')

In [ ]:
#example: variables (syllables) 5 and 9 are correlated in group 1 and 2, and variables 5 and 1 are artificially boosted in group 2
example3_group1=np.random.rand(n_mice,n_syllables)
example3_group2=np.random.rand(n_mice,n_syllables)
example3_group1[:,9]=example3_group1[:,5]*1.1 #variable 9 is 110% of variable 5
example3_group2[:,9]=example3_group2[:,5]*1.1 
example3_group2[:,5]+=.6
example3_group2[:,1]+=.7

In [ ]:
f, axes = plt.subplots(1,3)
f.set_size_inches(24,4)

example3_between,example3_within1,example3_within2=pc_dist(example3_group1,example3_group2)
sns.distplot(example3_between, color='purple',label=['distance between groups'],ax=axes[0])
sns.distplot(example3_within1, color='blue',label=['distance within group 1'],ax=axes[0])
sns.distplot(example3_within2, color='red',label=['distance within group 2'],ax=axes[0])
axes[0].set_title('pc distance')
axes[0].legend()

example3=contribution_of_each_syllable_to_distance_raw(example3_group1,example3_group2)
sns.barplot(x=np.array(range(n_syllables)),y=example3,ax=axes[1])
axes[1].set_title('contribution of variable to distance')
axes[1].axhline(cutoff, ls='--')

example3=contribution_of_each_syllable_to_distance(example3_group1,example3_group2)
sns.barplot(x=np.array(range(n_syllables)),y=example3,ax=axes[2])
axes[2].set_title('contribution of variable to distance after pca')
axes[2].axhline(cutoff, ls='--')

In [ ]:
#example: all variables (syllables) are artificially boosted in group 2
example3_group1=np.random.rand(n_mice,n_syllables)
example3_group2=np.random.rand(n_mice,n_syllables)+.75

In [ ]:
f, axes = plt.subplots(1,3)
f.set_size_inches(24,4)

example3_between,example3_within1,example3_within2=pc_dist(example3_group1,example3_group2)
sns.distplot(example3_between, color='purple',label=['distance between groups'],ax=axes[0])
sns.distplot(example3_within1, color='blue',label=['distance within group 1'],ax=axes[0])
sns.distplot(example3_within2, color='red',label=['distance within group 2'],ax=axes[0])
axes[0].set_title('pc distance')
axes[0].legend()

example3=contribution_of_each_syllable_to_distance_raw(example3_group1,example3_group2)
sns.barplot(x=np.array(range(n_syllables)),y=example3,ax=axes[1])
axes[1].set_title('contribution of variable to distance')
axes[1].axhline(cutoff, ls='--')

example3=contribution_of_each_syllable_to_distance(example3_group1,example3_group2)
sns.barplot(x=np.array(range(n_syllables)),y=example3,ax=axes[2])
axes[2].set_title('contribution of variable to distance after pca')
axes[2].axhline(cutoff, ls='--')